In [ ]:
from pathlib import Path

from tqdm import tqdm
import torch
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from fastsolv._module import _ALL_MODELS, _DESCRIPTOR_COLUMNS
from fastprop.data import standard_scale

# for compatibility with shap
for model in _ALL_MODELS:
    model.fnn.pop(0)
    model.fnn.pop(0)
# we remove the concatenation module, which we instead do manually here
# and the clamp module which is unsupported by SHAP

In [2]:
test_df = pd.read_csv(Path("../data/vermeire/solprop_fastprop_nonaq.csv"))

In [3]:
# need between 100 and 1000 samples to get a good estimate of the SHAP values
test_df = test_df.sample(n=2000, random_state=0)

In [4]:
inputs = torch.tensor(test_df[_DESCRIPTOR_COLUMNS + ["temperature"]].to_numpy(), dtype=torch.float32)
targets = torch.tensor(test_df["solubility"], dtype=torch.float32)
# we will use half of the data for 'integrating', and the other half for getting shap values
halfway_idx = inputs.shape[0] // 2

In [ ]:
# shap terminology explanation:
# background: 100 to 1000 random samples from the training data
# X: 100+ other samples
# run shap on each of these models, then average the results
per_model_shap = []
for model in tqdm(_ALL_MODELS, desc="Calculating SHAP values for each model"):
    # now scale and send the data to whatever device lightning put the model on
    X = inputs.to(model.device)
    solute_features, solvent_features, temperature = X[:, 0:1613], X[:, 1613:1613+1613], X[:, 1613+1613]  # batch 1 is solubility
    solute_features = standard_scale(solute_features, model.solute_means, model.solute_vars)
    solvent_features = standard_scale(solvent_features, model.solvent_means, model.solvent_vars)
    temperature = standard_scale(temperature, model.temperature_means, model.temperature_vars)
    scaled_X = torch.cat((solute_features, solvent_features, temperature[:, None]), dim=1)
    scaled_X.clamp_(min=-3.0, max=3.0)
    e = shap.DeepExplainer(model, scaled_X[:halfway_idx])
    model_shap_values = e.shap_values(scaled_X[halfway_idx:])
    # returns a list for multi-target problems, cast for uniformity
    if not isinstance(model_shap_values, list):
        model_shap_values = [model_shap_values]
    per_model_shap.append(model_shap_values)
all_shap_values = np.mean(np.array(per_model_shap), axis=0)

In [ ]:
# for each target, create a plot of the most important features
importance_threshold = 0.70
for i, target_name in enumerate(("logS", )):
    shap_values = all_shap_values[i].squeeze()
    # include features until the shap value is half the highest, aka half as important
    avg_shaps = np.mean(np.abs(shap_values), axis=0)
    avg_shaps, names, keep_idxs = zip(*sorted(zip(avg_shaps, _DESCRIPTOR_COLUMNS, list(range(len(avg_shaps)))), reverse=True))
    include_idx = 0
    for val in avg_shaps[1:]:
        if val > avg_shaps[0] * importance_threshold:
            include_idx += 1
        else:
            break
    explanation = shap.Explanation(values=shap_values[:, keep_idxs[:include_idx]], feature_names=names[0:include_idx])
    plt.cla()
    # _ = shap.plots.beeswarm(explanation, max_display=include_idx + 1, color_bar=False, color="shap_red", show=False)
    _ = shap.plots.bar(explanation, max_display=include_idx + 1, show=False)
    # _ = shap.plots.heatmap(explanation, max_display=include_idx + 1, show=False)
    out_fname = "shap.png"
    plt.savefig(out_fname, pad_inches=0.5, bbox_inches="tight")